<a href="https://colab.research.google.com/github/fwangliberty/AIoTDesign-Frontend/blob/master/cnn_small_cicids2018_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intrusion Detection by using small CICIDS 2018 DataSet

In [1]:
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt

## Step 1. Read cleaned CICIDS2018 small dataset

Connect to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# load train data
cleanfile = '/content/drive/My Drive/CICIDS2018/xtrain_small.csv'
labelfile = '/content/drive/My Drive/CICIDS2018/ytrain_small.csv'
xtestsmall = '/content/drive/My Drive/CICIDS2018/xtest_small.csv'
ytestsmall = '/content/drive/My Drive/CICIDS2018/ytest_small.csv'
SEED=2

In [4]:
X_train = pd.read_csv(cleanfile, skiprows=0,index_col=0) 
y_train = pd.read_csv(labelfile, skiprows=0,index_col=0)  
X_test = pd.read_csv(xtestsmall, skiprows=0, index_col=0) 
y_test = pd.read_csv(ytestsmall, skiprows=0,index_col=0) 
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
0,1755722.0,80.0,6.0,9743.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,205.275583,9743.000000,0.000000e+00,9743.0,9743.0,9743.0,9743.00,0.000000e+00,9743.0,9743.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,205.275583,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,225.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6156600.0,445.0,6.0,111085.0,6.0,5.0,455.0,338.0,140.0,0.0,75.833333,62.936211,145.0,0.0,67.600000,69.758870,7138.677589,99.023270,11108.500000,1.422834e+04,29284.0,4.0,111051.0,22210.20,1.245618e+04,29356.0,4.0,110986.0,27746.500000,1196.308349,29371.0,26744.0,0.0,0.0,0.0,0.0,132.0,112.0,54.012693,45.010577,0.0,145.0,66.083333,63.402693,4019.901515,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,72.090909,75.833333,67.600000,0.0,0.0,0.0,0.0,0.0,0.0,6.0,455.0,5.0,338.0,8192.0,0.0,4.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,242193.0,8080.0,6.0,10686.0,3.0,4.0,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,0.0,32.250000,53.767245,42579.075430,655.062699,1781.000000,3.912594e+03,9758.0,21.0,563.0,281.50,3.245620e+02,511.0,52.0,10196.0,3398.666667,5510.518064,9758.0,32.0,0.0,0.0,0.0,0.0,72.0,92.0,280.741157,374.321542,0.0,326.0,56.875000,115.406657,13318.696430,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,65.000000,108.666667,32.250000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,326.0,4.0,129.0,8192.0,219.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6934977.0,22.0,6.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,285714.285714,7.000000,0.000000e+00,7.0,7.0,0.0,0.00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,32.0,32.0,142857.142857,142857.142857,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,241.0,230.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4505141.0,80.0,6.0,5007525.0,5.0,3.0,646.0,364.0,646.0,0.0,129.200000,288.899983,364.0,0.0,121.333333,210.155498,201.696447,1.597596,715360.714286,1.868948e+06,4953524.0,16.0,5007525.0,1251881.25,2.467885e+06,4953524.0,406.0,3255.0,1627.500000,634.274783,2076.0,1179.0,0.0,0.0,0.0,0.0,112.0,72.0,0.998497,0.599098,0.0,646.0,112.222222,233.577491,54558.444444,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,126.250000,129.200000,121.333333,0.0,0.0,0.0,0.0,0.0,0.0,5.0,646.0,3.0,364.0,8192.0,221.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Here we can see the number of rows and columns for each table.
print(y_train.shape)

(30000, 1)


## Step 2. Normalization

The continuous feature values are normalized into the same feature space. This is important when using features that have different measurements, and is a general requirement of many machine learning algorithms. Therefore, the values for this dataset are also normalized using the Min-Max scaling technique, bringing them all within a range of [0,1].

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_train

array([[0.1888032 , 0.00122085, 0.35294118, ..., 0.        , 0.        ,
        0.        ],
       [0.66241015, 0.00679099, 0.35294118, ..., 0.        , 0.        ,
        0.        ],
       [0.02592255, 0.12330607, 0.35294118, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.3534946 , 0.83272799, 0.35294118, ..., 0.        , 0.        ,
        0.        ],
       [0.82190572, 0.00122085, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.51949634, 0.00679099, 0.35294118, ..., 0.        , 0.        ,
        0.        ]])

In [9]:
X_train.shape

(30000, 79)

In [10]:
X_test = scaler.fit_transform(X_test)

In [11]:
X_test.shape

(7500, 79)

y_train and y_test have to be one-hot-encoded. That means they must have dimension (number_of_samples, 15), where 15 denotes number of classes.

In [12]:
y_vector = y_train.values.tolist()
print(len(y_vector))
y_test_v = y_test.values.tolist()
print(len(y_test_v))

30000
7500


In [13]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_vector, 15)
y_test = to_categorical(y_test_v, 15)
print(y_train.shape)
print(y_test.shape)

(30000, 15)
(7500, 15)


## Step 3. Build the model

In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dense, Activation,Dropout
from tensorflow.keras.constraints import max_norm

In [130]:
#hyper-params
batch_size = 512 # increasing batch size with more gpu added

input_dim = X_train.shape[1]
num_class = 15                   # 15 intrusion classes, including benign traffic class
num_epochs = 90
learning_rates = 0.0001

optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
print(input_dim)
print(num_class)

79
15


In [16]:
#X_train_r = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train_r = np.zeros((len(X_train), input_dim, 1))
X_train_r[:, :, 0] = X_train[:, :input_dim]
print(X_train_r.shape)

(30000, 79, 1)


In [17]:
X_test_r = np.zeros((len(X_test), input_dim, 1))
X_test_r[:, :, 0] = X_test[:, :input_dim]
print(X_test_r.shape)

(7500, 79, 1)


In [135]:
model = Sequential()

# input layer
model.add(Conv1D(filters=64, kernel_size=13, padding='same', input_shape=(79,1)))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=5))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=256, kernel_size=5))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Conv1D(filters=512, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Conv1D(filters=256, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(num_class))
model.add(Activation('softmax'))


model.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_165 (Conv1D)          (None, 79, 64)            896       
_________________________________________________________________
batch_normalization_164 (Bat (None, 79, 64)            316       
_________________________________________________________________
activation_196 (Activation)  (None, 79, 64)            0         
_________________________________________________________________
max_pooling1d_53 (MaxPooling (None, 39, 64)            0         
_________________________________________________________________
conv1d_166 (Conv1D)          (None, 35, 128)           41088     
_________________________________________________________________
batch_normalization_165 (Bat (None, 35, 128)           140       
_________________________________________________________________
activation_197 (Activation)  (None, 35, 128)         

In [136]:
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy']) 

## Step 4. Training the model

In [137]:
# fit network
model.fit(X_train_r, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test_r, y_test), verbose=1)

Epoch 1/90
59/59 [==============================] - 4s 40ms/step - loss: 1.4792 - accuracy: 0.5338 - val_loss: 2.9308 - val_accuracy: 0.0667
Epoch 2/90
59/59 [==============================] - 2s 28ms/step - loss: 0.2848 - accuracy: 0.8891 - val_loss: 3.3144 - val_accuracy: 0.0667
Epoch 3/90
59/59 [==============================] - 2s 28ms/step - loss: 0.2275 - accuracy: 0.9047 - val_loss: 3.4880 - val_accuracy: 0.0820
Epoch 4/90
59/59 [==============================] - 2s 28ms/step - loss: 0.2112 - accuracy: 0.9110 - val_loss: 2.8456 - val_accuracy: 0.0947
Epoch 5/90
59/59 [==============================] - 2s 29ms/step - loss: 0.1943 - accuracy: 0.9177 - val_loss: 2.1487 - val_accuracy: 0.2108
Epoch 6/90
59/59 [==============================] - 2s 28ms/step - loss: 0.1792 - accuracy: 0.9278 - val_loss: 1.3835 - val_accuracy: 0.5365
Epoch 7/90
59/59 [==============================] - 2s 29ms/step - loss: 0.1703 - accuracy: 0.9301 - val_loss: 0.9040 - val_accuracy: 0.6808
Epoch 8/90
59

In [81]:
# evaluate model
accuracy = model.evaluate(X_test_r, y_test, batch_size=batch_size, verbose=1)

15/15 [==============================] - 0s 11ms/step - loss: 0.5118 - accuracy: 0.8867
